A basic momentum strategy

In [9]:
import pandas as pd

from quant.metric import sharpe_ratio

# Parameters of our strategy
univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','XRPUSDT','DOTUSDT','MATICUSDT']
data_period = 4 # in hours
train_start = '2020-01-01'
train_end = '2022-12-31'
test_start = '2023-01-01'
test_end = '2023-12-31'

freq = f'{data_period}h'
train_filename = f'../../data/Abel/{freq}_{train_start}_{train_end}'

train_returns = pd.read_csv(train_filename + '.csv', index_col=0, parse_dates=True)
train_returns.head(3)

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT
open-time,,,,
2019-12-31 19:00:00,NaN,NaN,NaN,NaN
2019-12-31 23:00:00,-0.003487,0.002612,0.003628,-0.003699
2020-01-01 03:00:00,-0.001346,0.002452,0.000301,-0.003531


In [10]:
from quant.algorithms.classic.momentum import rolling_avg_rank
from quant.metric import strategy_returns

# For each of the following multiples of data_period, we make a momentum strategy and calculate the strategy return:
horizons = list(range(1, 7))
strats_returns = {}
for h in horizons:
    # Calculate the portfolio weights for the strategy
    weights = rolling_avg_rank(train_returns, h)
    strats_returns[h] = strategy_returns(weights, train_returns)
strats_returns = pd.DataFrame(strats_returns)
strats_returns.head(3)

,1,2,3,4,5,6
open-time,,,,,,
2019-12-31 19:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-12-31 23:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-01 03:00:00,0.001912,0.001912,0.001912,0.001912,0.001912,0.001912


In [11]:
# Calculate the annualized Sharpe of each strategy
sr = sharpe_ratio(strats_returns, data_period)
horizon_ratio_trained = sr.idxmax()
print(f'The trained horizon-to-period ratio is {horizon_ratio_trained}, so the corresponding horizon is {horizon_ratio_trained*data_period} hours.')
sr

The trained horizon-to-period ratio is 5, so the corresponding horizon is 20 hours.


1   -2.060550
2   -0.653345
3    0.429198
4    0.902171
5    1.662567
6    0.571180
dtype: float64

# Test the strategy

In [12]:
test_filename = f'../../data/Abel/{freq}_{test_start}_{test_end}'

test_returns = pd.read_csv(test_filename + '.csv', index_col=0, parse_dates=True)
test_returns.head(3)

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT
open-time,,,,
2022-12-31 19:00:00,NaN,NaN,NaN,NaN
2022-12-31 23:00:00,0.000410,0.000946,-0.000326,-0.004892
2023-01-01 03:00:00,0.001353,0.000711,0.002937,0.001229


In [13]:
# Apply the trained strategy to the test data
weights = rolling_avg_rank(test_returns, horizon_ratio_trained)
# Calculate the return of the strategy over each test period
strat_returns = strategy_returns(weights, test_returns)
strat_returns.head(3)

open-time
2022-12-31 19:00:00    0.000000
2022-12-31 23:00:00    0.000000
2023-01-01 03:00:00   -0.000392
dtype: float64

In [14]:
sr = sharpe_ratio(strat_returns, data_period)
print(f'The Sharpe ratio of the strategy on the test data is {sr:.3f}')

The Sharpe ratio of the strategy on the test data is 0.031


In [15]:
strat_returns_cumulative = strat_returns.cumsum()
print(strat_returns_cumulative)

open-time
2022-12-31 19:00:00    0.000000
2022-12-31 23:00:00    0.000000
2023-01-01 03:00:00   -0.000392
2023-01-01 07:00:00   -0.000106
2023-01-01 11:00:00    0.002150
                         ...   
2023-12-30 03:00:00    0.004453
2023-12-30 07:00:00    0.001501
2023-12-30 11:00:00    0.000529
2023-12-30 15:00:00    0.003529
2023-12-30 19:00:00    0.006072
Length: 2184, dtype: float64


In [16]:
import importlib
import quant.metric
importlib.reload(quant.metric)

from quant.metric import max_drawdown_stats

stats = max_drawdown_stats(strat_returns_cumulative)
stats

{'value': np.float64(-inf),
 'peak_date': Timestamp('2022-12-31 19:00:00'),
 'trough_date': Timestamp('2023-01-01 03:00:00'),
 'recovery_date': Timestamp('2023-01-01 11:00:00'),
 'running_drawdown': open-time
 2022-12-31 19:00:00         NaN
 2022-12-31 23:00:00         NaN
 2023-01-01 03:00:00        -inf
 2023-01-01 07:00:00        -inf
 2023-01-01 11:00:00    0.000000
                          ...   
 2023-12-30 03:00:00   -0.963326
 2023-12-30 07:00:00   -0.987633
 2023-12-30 11:00:00   -0.995643
 2023-12-30 15:00:00   -0.970929
 2023-12-30 19:00:00   -0.949986
 Length: 2184, dtype: float64}